Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# 05. Train in Spark
* Create Workspace
* Create Experiment
* Copy relevant files to the script folder
* Configure and Run

## Prerequisites
Make sure you go through the [00. Installation and Configuration](00.configuration.ipynb) Notebook first if you haven't.

In [ ]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

## Initialize Workspace

Initialize a workspace object from persisted configuration.

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

## Create Experiment


In [ ]:
experiment_name = 'train-on-remote-vm'
script_folder = './samples/train-on-remote-vm'

import os
os.makedirs(script_folder, exist_ok = True)

from azureml.core import Experiment

exp = Experiment(workspace = ws, name = experiment_name)

## Copy files


Copy `train-spark.py` and `iris.csv` into the project folde

In [ ]:
from shutil import copyfile

# copy iris dataset in to project folder
copyfile('iris.csv', os.path.join(script_folder, 'iris.csv'))

# copy train-spark.py file into project folder
# train-spark.py trains a simple LogisticRegression model using Spark.ML algorithm
copyfile('train-spark.py', os.path.join(script_folder, 'train-spark.py'))

Review the train-spark.py file in the project folder.

In [ ]:
with open(os.path.join(project_folder, 'train-spark.py'), 'r') as fin:
    print(fin.read())

## Configure & Run

### Attach an HDI cluster
To use HDI commpute target:
 1. Create an Spark for HDI cluster in Azure. Here is some [quick instructions](https://docs.microsoft.com/en-us/azure/machine-learning/desktop-workbench/how-to-create-dsvm-hdi). Make sure you use the Ubuntu flavor, NOT CentOS.
 2. Enter the IP address, username and password below

In [ ]:
from azureml.core.compute import HDInsightCompute

try:
    # if you want to connect using SSH key instead of username/password you can provide parameters private_key_file and private_key_passphrase
    hdi_compute_new = HDInsightCompute.attach(ws, 
                             name="hdi-attach", 
                             address="hdi-ignite-demo-ssh.azurehdinsight.net", 
                             ssh_port=22, 
                             username='<username>', 
                             password='<password>')

except UserErrorException as e:
    print("Caught = {}".format(e.message))
    print("Compute config already attached.")
    
    
hdi_compute_new.wait_for_completion(show_output=True)

### Configure HDI run

In [ ]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies


# Load the "cpu-dsvm.runconfig" file (created by the above attach operation) in memory
run_config = RunConfiguration(framework = "python")

# Set compute target to the Linux DSVM
run_config.target = hdi_compute.name

# Use Docker in the remote VM
# run_config.environment.docker.enabled = True

# Use CPU base image from DockerHub
# run_config.environment.docker.base_image = azureml.core.runconfig.DEFAULT_CPU_IMAGE
# print('Base Docker image is:', run_config.environment.docker.base_image)

# Ask system to provision a new one based on the conda_dependencies.yml file
run_config.environment.python.user_managed_dependencies = False

# Prepare the Docker and conda environment automatically when executingfor the first time.
# run_config.prepare_environment = True

# specify CondaDependencies obj
# run_config.environment.python.conda_dependencies = CondaDependencies.create(conda_packages=['scikit-learn'])
# load the runconfig object from the "myhdi.runconfig" file generated by the attach operaton above.

### Submit the script to HDI

In [ ]:
script_run_config = ScriptRunConfig(source_directory = project.project_directory,
                                    script= 'train-spark.py',
                                    run_config = run_config)
run = experiment.submit(script_run_config)

run.wait_for_completion(show_output = True)

In [ ]:
# get the URL of the run history web page
print(helpers.get_run_history_url(run))

In [ ]:
# get all metris logged in the run
metrics = run.get_metrics()
print(metrics)